In [ ]:
import torch

if torch.cuda.is_available():
    device = torch.device('cuda')  # CUDA GPU
elif torch.backends.mps.is_available():
    device = torch.device('mps') #Apple GPU
else:
    device = torch.device("cpu")

print(device)

In [ ]:
print("Python")

In [2]:
# Step 1: Import necessary libraries and modules
import sys
import torch

# Add the path to the directory where backbone.py is located
sys.path.append('/data/cmpe258-sp24/013978029/DeepDataMiningLearning/hw1')  # Add the folder containing backbone.py

# Import the backbone functions and classes
from backbone import get_efficientnet_backbone, CustomBackboneWithFPN

In [ ]:
backbone = get_efficientnet_backbone(pretrained=True)

# Print all nested layers with full details to find exact layer names
def print_all_layer_names_with_details(model, parent_name=""):
    for name, module in model.named_children():
        full_name = f"{parent_name}.{name}" if parent_name else name
        print(full_name)
        print_all_layer_names_with_details(module, full_name)

print("Detailed layer structure of EfficientNet-B0:")
print_all_layer_names_with_details(backbone)


In [ ]:
# Function to check if specified layers exist in the model
def check_layers_exist(model, layer_names):
    existing_layers = set(name for name, _ in model.named_modules())
    missing_layers = [layer for layer in layer_names if layer not in existing_layers]
    
    if missing_layers:
        print("The following layers specified in return_layers do not exist in the model:")
        for layer in missing_layers:
            print(f" - {layer}")
    else:
        print("All specified layers in return_layers exist in the model.")

# Initialize the EfficientNet-B0 backbone with pretrained weights
backbone = get_efficientnet_backbone(pretrained=True)

# Specify the layers to check
return_layers = {
            "0.1": "0",   # Early stage feature extraction
            "0.2": "1",   # Mid-level feature extraction
            "0.4": "2",   # Deeper stage feature extraction
            "0.6": "3"    # Deepest stage before the final layer
}

# Check if the layers in return_layers exist in the backbone model
check_layers_exist(backbone, return_layers.keys())


In [ ]:
# Step 2: Initialize the model
# Get the EfficientNet-B0 backbone with pretrained weights
backbone = get_efficientnet_backbone(pretrained=True)

# Specify the layer names to hook based on your model structure
layer_names = [
    "0.1.0.block",
    "0.2.1.block",
    "0.4.2.block",
    "0.6.3.block"
]

# Create an instance of CustomBackboneWithFPN (no trainable_layers argument)
model = CustomBackboneWithFPN(backbone, layer_names=layer_names, out_channels=256)

# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [ ]:
# Step 3: Create a test input tensor
x = torch.rand(1, 3, 256, 256).to(device)  # Adjust input size as needed

# Step 4: Run a forward pass and print the output shapes
model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Disable gradient calculation for testing
    output = model(x)

# Print output shapes
print("Output feature maps:")
for name, feature_map in output.items():
    print(f"Layer {name}: {feature_map.shape}")

In [ ]:
import os

path = "/data/cmpe258-sp24/013978029/dataset/Kitti"
if os.path.exists(path):
    print("Path exists!")
else:
    print("Path does not exist.")

In [ ]:
import os

# Paths to the directories containing the training and testing images
train_image_dir = '/data/cmpe258-sp24/013978029/dataset/Kitti/raw/training/image_2'
test_image_dir = '/data/cmpe258-sp24/013978029/dataset/Kitti/testing/image_2/'

# Generate the list of image IDs (filenames without extensions) for training
train_image_ids = [f.split('.')[0] for f in os.listdir(train_image_dir) if f.endswith('.png')]

# Generate the list of image IDs (filenames without extensions) for validation
val_image_ids = [f.split('.')[0] for f in os.listdir(test_image_dir) if f.endswith('.png')]

# Create the ImageSets directory if it doesn't exist
os.makedirs('/data/cmpe258-sp24/013978029/dataset/Kitti/ImageSets', exist_ok=True)

# Write to train.txt
with open('/data/cmpe258-sp24/013978029/dataset/Kitti/ImageSets/train.txt', 'w') as train_file:
    for image_id in sorted(train_image_ids):
        train_file.write(f"{image_id}\n")

# Write to val.txt
with open('/data/cmpe258-sp24/013978029/dataset/Kitti/ImageSets/val.txt', 'w') as val_file:
    for image_id in sorted(val_image_ids):
        val_file.write(f"{image_id}\n")

print("train.txt and val.txt have been created successfully.")

In [ ]:
import sys
sys.path.append('/data/cmpe258-sp24/013978029/DeepDataMiningLearning/DeepDataMiningLearning/detection')  # Adjust this to your correct path

# Import necessary functions and classes
from dataset import get_dataset

# Simulate argparse arguments
class Args:
    data_path = '/data/cmpe258-sp24/013978029/dataset/Kitti'  # Replace with your KITTI dataset path
    annotationfile = ''
    data_augmentation = 'hflip'
    backend = 'PIL'
    use_v2 = False
    weights = None
    test_only = False

args = Args()

# Load and print a sample
dataset, _ = get_dataset('kitti', is_train=True, is_val=False, args=args)
sample_image, sample_target = dataset[0]

print("Sample Image Shape:", sample_image.size)
print("Sample Target:", sample_target)


In [ ]:
import sys
sys.path.append('/data/cmpe258-sp24/013978029/DeepDataMiningLearning/DeepDataMiningLearning/detection')

from dataset_kitti import MyKittiDetection, get_transformsimple

# Create the dataset instance
dataset = MyKittiDetection(root='/data/cmpe258-sp24/013978029/dataset/Kitti', train=True, transform=get_transformsimple(train=True))

# Iterate over a few examples to check the output
for i in range(3):  # Adjust the range as needed to test more images
    img, target = dataset[i]
    # The debug statements in `get_transformsimple` will print the image sizes


In [ ]:
import sys
sys.path.append('/data/cmpe258-sp24/013978029/DeepDataMiningLearning')  # Adjust this to your correct path

# Import necessary functions and classes
import torch
from DeepDataMiningLearning.detection.dataset import get_dataset
import torchvision.transforms.functional as F

# Simulate argparse arguments
class Args:
    data_path = '/data/cmpe258-sp24/013978029/dataset/Kitti'  # Replace with your KITTI dataset path
    annotationfile = ''
    data_augmentation = 'hflip'
    backend = 'PIL'
    use_v2 = False
    weights = None
    test_only = False

args = Args()

# Load the dataset
dataset, _ = get_dataset('kitti', is_train=True, is_val=False, args=args)

# Iterate through the dataset and print the size of each image
for idx in range(len(dataset)):
    image, target = dataset[idx]
    if isinstance(image, torch.Tensor):
        image = F.to_pil_image(image)  # Convert Tensor to PIL Image for getting the size
    print(f"Image {idx + 1} Size: {image.size}")  # Print (width, height)

In [ ]:
import sys
sys.path.append('/data/cmpe258-sp24/013978029/DeepDataMiningLearning')  # Ensure the correct path is set

# Import the get_dataset function from your dataset module
from DeepDataMiningLearning.detection.dataset import get_dataset

# Simulate the argparse arguments if needed
class Args:
    data_path = '/data/cmpe258-sp24/013978029/dataset/Kitti'  # Your dataset path
    annotationfile = ''
    dataset = 'kitti'
    data_augmentation = 'hflip'
    backend = 'PIL'
    use_v2 = False

args = Args()

# Load the dataset with transformations applied
dataset, _ = get_dataset('kitti', is_train=True, is_val=False, args=args)

# Iterate through the dataset and print each image's transformed size
print("Verifying transformed image sizes...")
consistent = True
target_size = (3, 375, 1242)  # Replace with your target size if different

for idx in range(len(dataset)):
    image, _ = dataset[idx]
    print(f"Image {idx + 1} Transformed Size: {image.shape}")
    
    if image.shape != target_size:
        print(f"Inconsistent image size found: {image.shape} at index {idx}")
        consistent = False

if consistent:
    print("All images have consistent sizes.")
else:
    print("Some images have inconsistent sizes. Check the output for details.")


In [ ]:
import sys
import torch
sys.path.append('/data/cmpe258-sp24/013978029/DeepDataMiningLearning') 

from DeepDataMiningLearning.detection.transforms import Resize, ToDtype, Compose
from DeepDataMiningLearning.detection.dataset_kitti import KittiDataset

# Remove PILToTensor as Resize now returns a tensor directly
transform = Compose([Resize((370, 1224)), ToDtype(torch.float, scale=True)])
dataset = KittiDataset(root="/data/cmpe258-sp24/013978029/dataset/Kitti", train=True, transform=transform)

for i in range(3):
    img, target = dataset[i]
    print(f"[TEST] Image Shape: {img.shape}")
    print(f"[TEST] Target Keys: {target.keys()}")

In [ ]:
import os

# Define your dataset path
data_path = '/data/cmpe258-sp24/013978029/dataset/Kitti'
image_dir = os.path.join(data_path, 'raw/training/image_2')
label_dir = os.path.join(data_path, 'raw/training/label_2')

# Check if the required directories exist
def check_directory_structure():
    if not os.path.exists(image_dir):
        print(f"Image directory missing: {image_dir}")
    else:
        print(f"Image directory exists: {image_dir}")
    
    if not os.path.exists(label_dir):
        print(f"Label directory missing: {label_dir}")
    else:
        print(f"Label directory exists: {label_dir}")

# Check if the first few files exist
def check_sample_files(num_samples=20):
    missing_images = []
    missing_labels = []
    for i in range(num_samples):
        image_path = os.path.join(image_dir, f"{i:06d}.png")
        label_path = os.path.join(label_dir, f"{i:06d}.txt")
        
        if not os.path.isfile(image_path):
            missing_images.append(image_path)
        
        if not os.path.isfile(label_path):
            missing_labels.append(label_path)
    
    if missing_images:
        print(f"Missing {len(missing_images)} image files:")
        for path in missing_images:
            print(f" - {path}")
    else:
        print("All expected image files are present.")

    if missing_labels:
        print(f"Missing {len(missing_labels)} label files:")
        for path in missing_labels:
            print(f" - {path}")
    else:
        print("All expected label files are present.")

# Run the checks
print("Checking dataset directory structure...")
check_directory_structure()

print("\nChecking sample image and label files...")
check_sample_files(num_samples=5)

In [ ]:
import os

# Path to the directory we want to verify
image_dir = '/data/cmpe258-sp24/013978029/dataset/Kitti/raw/testing/image_2'

# Check if the directory exists
if os.path.isdir(image_dir):
    print(f"Directory exists: {image_dir}")
    
    # List all files in the directory
    files = os.listdir(image_dir)
    
    # Filter to only include .png files
    png_files = [f for f in files if f.endswith('.png')]
    
    if png_files:
        # Sort files to check the sequence
        png_files.sort()

        # Check the naming format and sequence
        all_sequential = True
        for i, file_name in enumerate(png_files):
            expected_name = f"{i:06}.png"
            if file_name != expected_name:
                print(f"File {file_name} does not match expected name {expected_name}.")
                all_sequential = False
                break
        
        if all_sequential:
            print("All images are present and follow the expected naming format.")
        else:
            print("Some images are missing or incorrectly named.")
    else:
        print("No PNG image files found in the directory.")
else:
    print(f"Directory does not exist: {image_dir}")

In [1]:
import sys
sys.path.append('/data/cmpe258-sp24/013978029/DeepDataMiningLearning') 

# Import necessary functions and classes from mytrain.py
from DeepDataMiningLearning.detection.mytrain import get_args_parser, main

# Simulate the argparse arguments
class Args:
    data_path = '/data/cmpe258-sp24/013978029/dataset/Kitti'  # Your dataset path
    annotationfile = ''
    dataset = 'kitti'
    model = 'custom_fpn'
    trainable = 0
    device = 'cuda'  # Use 'cpu' if not running on a GPU
    batch_size = 4  # Reduced batch size
    epochs = 1  # Set to a small number for a test
    saveeveryepoch = 1
    workers = 4
    opt = 'sgd'
    lr = 0.02
    momentum = 0.9
    weight_decay = 1e-4
    lr_scheduler = 'multisteplr'
    lr_steps = [16, 22]
    lr_gamma = 0.1
    print_freq = 5
    output_dir = './output'
    resume = ''
    start_epoch = 0
    aspect_ratio_group_factor = -1
    rpn_score_thresh = None
    data_augmentation = 'hflip'
    sync_bn = False
    test_only = False
    use_deterministic_algorithms = False
    multigpu = False
    world_size = 4
    dist_url = 'env://'
    weights = None
    weights_backbone = None
    amp = False
    backend = 'PIL'
    use_v2 = False
    expname = 'experiment'
    max_batches = 5  # Limit the number of batches for testing

# Instantiate the arguments object
args = Args()

# Run the training/test process
main(args)

/home/013978029/.conda/envs/cmpe249project/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading data
train set len: 4064
test set len: 4065
Creating data loaders
Creating model


/home/013978029/.conda/envs/cmpe249project/lib/python3.12/site-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Start training
Checking image path: /data/cmpe258-sp24/013978029/dataset/Kitti/raw/training/image_2/001195.pngChecking image path: /data/cmpe258-sp24/013978029/dataset/Kitti/raw/training/image_2/000245.png

Checking image path: /data/cmpe258-sp24/013978029/dataset/Kitti/raw/training/image_2/001119.png
[DEBUG] get_label for 000245 returns 5 objects.
Checking image path: /data/cmpe258-sp24/013978029/dataset/Kitti/raw/training/image_2/002554.png
[DEBUG] Converting target for image_id 000245: [{'image_id': '000245', 'type': 'Car', 'truncated': 0.39, 'occluded': 1, 'alpha': 2.49, 'bbox': [871.24, 138.44, 1241.0, 369.5], 'dimensions': [1.85, 1.68, 4.34], 'location': [4.89, 1.58, 6.76], 'rotation_y': 3.1}, {'image_id': '000245', 'type': 'Cyclist', 'truncated': 0.0, 'occluded': 0, 'alpha': -1.59, 'bbox': [593.67, 172.0, 612.03, 219.1], 'dimensions': [1.71, 0.63, 2.12], 'location': [-0.32, 1.7, 27.39], 'rotation_y': -1.6}, {'image_id': '000245', 'type': 'Car', 'truncated': 0.48, 'occluded': 0, 

RuntimeError: The size of tensor a (612) must match the size of tensor b (306) at non-singleton dimension 3